In [1]:
# !pip install optuna

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
data=pd.read_excel('./data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   int64         
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   int64         
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   int64         
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [4]:
data = data[data["Date"].isin(pd.date_range('2016-01-04', '2019-11-30'))]
data.reset_index(drop=True, inplace=True)

In [5]:
data = data[['Date', 'REV OBD']]

scaler = MinMaxScaler()
scale_cols = ['REV OBD']

# 예측 데이터 사이즈
max_prediction_length = 20

In [6]:
# 학습용 데이터
# 전체 데이터의 70%만 사용
data_p = data.iloc[:int(data.index.max() * .7), :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [7]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
actual_data = data.loc[~data.index.isin(data_p.index)][scale_cols]
actual_data.shape

(429, 1)

In [8]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

In [9]:
x, y = sliding_windows(training_data, 60, 10)

In [21]:
y[:,1,:]

array([[1.99363326e-01],
       [2.49069919e-01],
       [1.78191999e-01],
       [3.06830821e-04],
       [9.66900625e-02],
       [1.93802017e-01],
       [2.80289955e-01],
       [1.82142446e-01],
       [2.58926859e-01],
       [1.47854102e-01],
       [5.92950562e-02],
       [1.29559314e-01],
       [1.84865570e-01],
       [2.74997123e-01],
       [1.75737353e-01],
       [2.49300042e-01],
       [1.48697887e-01],
       [6.52015495e-04],
       [7.51735512e-02],
       [1.62006674e-01],
       [2.46346796e-01],
       [1.53261995e-01],
       [2.19384037e-01],
       [1.82410923e-01],
       [1.91769263e-02],
       [1.19970851e-01],
       [1.97407280e-01],
       [2.68707092e-01],
       [1.44018717e-01],
       [2.81977525e-01],
       [1.22118667e-01],
       [8.22306601e-02],
       [1.51766195e-01],
       [2.73194492e-01],
       [3.58915353e-01],
       [2.89226403e-01],
       [4.07087792e-01],
       [2.35569363e-01],
       [1.57365857e-01],
       [3.63709585e-01],


In [11]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        # print(f'num_layer = {self.num_layers}, hidden_size = {self.hidden_size}, h_out shape = {h_out.shape}')
        out = self.fc(h_out)
        # print('output shape = ', len(out))
        return out

In [12]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [63]:

def objective(trial):

    cfg = { 
            # 'device' : "cuda" if torch.cuda.is_available() else "cpu", # 파라미터라고 볼 수 없음
            'batch_size' : trial.suggest_categorical('batch_size',[64, 128, 256, 512]), # [64, 128, 256]
            # 'num_epochs' : trial.suggest_int('num_epochs', 10, 100, 10), #1000, 1600, 100), # 충분히 학습되는 것이 좋으므로 파라미터라고 볼 수 없음
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용          
            # 'momentum': trial.suggest_uniform('momentum', 0.4, 0.99), # optimizer에서 사용하는 옵션이지만, 코드에서는 사용하지 않음
            # 'optimizer': trial.suggest_categorical('optimizer',[optim.SGD, optim.Adam]), # trial.suggest_categorical('optimizer',[optim.SGD, optim.Adam]),
            # 'activation': trial.suggest_categorical('activation',[ torch.nn.relu6, torch.nn.relu ]), # 사용되지 않은 parameter
            'hidden_size': trial.suggest_categorical('hidden_size',[16, 32, 64, 128, 256, 512, 1024]),
            'num_layers': trial.suggest_int('num_layers', 1, 5, 1),       
            # 'num_classes': trial.suggest_int('num_classes', 1, 1), # 실수를 예측하므로 고정된 값으로 사용
        }

    torch.manual_seed(42)

    lookback_length = 60
    forecast_length = max_prediction_length
    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정
    num_epochs = 10 # parameter에서 빼서 상수로 설정
    x, y = sliding_windows(training_data, lookback_length, forecast_length)

    train_dl, val_dl, input_size = get_data_loader(x, y[:, 0, :],  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, num_epochs)

    # print('best loss for the trial = ', best_loss)
    predict_data = []
    actual_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = x  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정
   
    # print('-----------------------y shape before loop = ', y.shape)
    for j, i in enumerate(range(max_prediction_length - 1)):

        # feed the last forecast back to the model as an input
        # print(f'j = {j}')
        # print(f'y shape = {y.shape}')
        # print(f'Before update data = {x_pred.shape} & y_pred = {y_pred.shape}, expand_dim shape = {np.expand_dims(y[:, j, :], 1).shape}')
        x_pred = np.append(x_pred[:, 1:, :], np.expand_dims(y[:, j, :], 1), axis=1)
        # print(f'After update data = {x_pred.shape}')
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)

        # print(f'model result yt_pred = {yt_pred.shape}')
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)
        # save actual data
        actual_data.append(y[:, j, :])

    # print(f'After loop predict_data = {len(predict_data)} & predict_data = {len(predict_data)}')
    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    # predict_data = scaler.inverse_transform(predict_data) # actual_data는 scale되어 있는 데이터임
    actual_data = np.array(actual_data).reshape(-1, 1)
    
    # print(f'predict_data = {predict_data[:3]}, actual data = {actual_data[:3]}')
    # print(f'actual size = {actual.shape}, predict = {predict_data.shape}')
    SMAPE = smape(actual_data, predict_data)
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [64]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 20)

[I 2022-06-16 22:51:15,144] A new study created in memory with name: no-name-11b4f8a9-f604-4509-b41c-3e1ac90fccad



 Epoch 0 	 Training Loss: 38.54040144383907 	 Validation Loss: 62.71350860595703 

Validation Loss Decreased(inf--->62.713509) 	 Saving The Model
Validation Loss Decreased(62.713509--->19.083183) 	 Saving The Model
Validation Loss Decreased(19.083183--->9.727250) 	 Saving The Model
Validation Loss Decreased(9.727250--->6.433501) 	 Saving The Model
Validation Loss Decreased(6.433501--->2.297514) 	 Saving The Model

 Epoch 5 	 Training Loss: 4.421632528305054 	 Validation Loss: 6.299924850463867 

Validation Loss Decreased(2.297514--->0.214509) 	 Saving The Model


[I 2022-06-16 22:51:17,432] Trial 0 finished with value: 165.4524595857317 and parameters: {'batch_size': 512, 'learning_rate': 0.03697182649812958, 'hidden_size': 256, 'num_layers': 4}. Best is trial 0 with value: 165.4524595857317.


 
SMAPE : 165.4524595857317

 Epoch 0 	 Training Loss: 0.06874793364355962 	 Validation Loss: 0.03677862882614136 

Validation Loss Decreased(inf--->0.036779) 	 Saving The Model
Validation Loss Decreased(0.036779--->0.031350) 	 Saving The Model
Validation Loss Decreased(0.031350--->0.022192) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.025320327530304592 	 Validation Loss: 0.02643493562936783 



[I 2022-06-16 22:51:18,888] Trial 1 finished with value: 37.75123570296953 and parameters: {'batch_size': 128, 'learning_rate': 0.0027876331160936138, 'hidden_size': 64, 'num_layers': 5}. Best is trial 1 with value: 37.75123570296953.


...

 
SMAPE : 35.65748634671147

 Epoch 0 	 Training Loss: 0.0845626915494601 	 Validation Loss: 0.024731244891881943 

Validation Loss Decreased(inf--->0.024731) 	 Saving The Model
Validation Loss Decreased(0.024731--->0.019213) 	 Saving The Model


[I 2022-06-16 22:52:09,390] Trial 18 finished with value: 30.53173139099194 and parameters: {'batch_size': 256, 'learning_rate': 0.015507603161339997, 'hidden_size': 32, 'num_layers': 1}. Best is trial 18 with value: 30.53173139099194.


Validation Loss Decreased(0.019213--->0.017032) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.02199672597150008 	 Validation Loss: 0.016890130937099457 

Validation Loss Decreased(0.017032--->0.016890) 	 Saving The Model
Validation Loss Decreased(0.016890--->0.016414) 	 Saving The Model
 
SMAPE : 30.53173139099194

 Epoch 0 	 Training Loss: 0.07045551389455795 	 Validation Loss: 0.04768942482769489 

Validation Loss Decreased(inf--->0.047689) 	 Saving The Model
Validation Loss Decreased(0.047689--->0.030311) 	 Saving The Model
Validation Loss Decreased(0.030311--->0.021099) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.02554055241247018 	 Validation Loss: 0.026631860062479973 



[I 2022-06-16 22:52:10,144] Trial 19 finished with value: 35.953759023534275 and parameters: {'batch_size': 128, 'learning_rate': 0.015498810013577778, 'hidden_size': 32, 'num_layers': 2}. Best is trial 18 with value: 30.53173139099194.


 
SMAPE : 35.953759023534275


In [32]:
np.expand_dims(y[:, 9, :], 1).shape

(929, 1, 1)